## This is playground repo to play around with local LLM

In [1]:
import ollama

In [2]:
MODEL_NAME = "phi3" # will try this one first then maybe "qwen:3b"
ollama.pull(MODEL_NAME)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [3]:
def generate_response(prompt: str) -> str:
    response = ollama.chat(
        model = MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        options={
            "temperature": 0.7,
            "num_predict": 512,
        }
    )
    return response['message']['content']

In [4]:
print(generate_response("Hello Ollama, how are you today?"))

I'm just a string of code here with no feelings or physical form, so I don't experience days the way humans do. However, if I were to simulate an answer: "I'm functioning optimally and ready for our interaction! How can I help you today?"


### Let's try streaming!

In [6]:
def generate_streaming(prompt: str):
    stream = ollama.chat(
        model = MODEL_NAME,
        messages = [{
            "role": "user",
            "content": prompt
        }],
        stream=True,
        options={
            "temparature": 0.7,
            "num_predict": 256
        }
    )
    
    full = []
    for chunk in stream:
        content = chunk['message']['content']
        print(content, end="", flush=True)
        full.append(content)
    print()
    return "".join(full)

If you run it now, you can see how tokens are returning as they are generated, not all at once in the end, this is important feature for UX, and most of chat interfaces use it.

In [7]:
generate_streaming('Why sky is blue?')

The color of the sky appears predominantly blue due to a phenomenon called Rayleigh scattering. This effect occurs when molecules and small particles in Earth's atmosphere interact with sunlight, causing shorter wavelength light (blue) to scatter more than longer wavelength light (red). Since our eyes are most sensitive to the green-yellow part of the spectrum, we perceive that color as being "whitish" or blue.

When there are fewer particles in the atmosphere, such as on clear sunny days with little pollution and moisture content—like during a dry day without rain clouds present —the sky appears even bluer because less light is absorbed before it reaches our eyes. However, when there'selavent occurs more atmospheric particulates like dust or water droplets from cloud formation due to humidity increases the scattering and reflection of sunlight in all directions which can lead to a whiter sky with possible pinkish undertones during events such as halos around the sun.

During twilight 

'The color of the sky appears predominantly blue due to a phenomenon called Rayleigh scattering. This effect occurs when molecules and small particles in Earth\'s atmosphere interact with sunlight, causing shorter wavelength light (blue) to scatter more than longer wavelength light (red). Since our eyes are most sensitive to the green-yellow part of the spectrum, we perceive that color as being "whitish" or blue.\n\nWhen there are fewer particles in the atmosphere, such as on clear sunny days with little pollution and moisture content—like during a dry day without rain clouds present —the sky appears even bluer because less light is absorbed before it reaches our eyes. However, when there\'selavent occurs more atmospheric particulates like dust or water droplets from cloud formation due to humidity increases the scattering and reflection of sunlight in all directions which can lead to a whiter sky with possible pinkish undertones during events such as halos around the sun.\n\nDuring tw

### Let's try simple chat loop!

In [ ]:
import ollama

MODEL_NAME = "phi3"  # You can change to another model if you have it pulled locally


def chat_loop(model=MODEL_NAME, temperature=1.1, num_predict=256):
    history = []
    print(f"Starting chat with {model}. Type 'exit' to quit.")

    while True:
        try:
            user_input = input("> ")
        except (KeyboardInterrupt, EOFError):
            print("\nending chat...")
            break

        if user_input.strip().lower() in {"exit", "\\q", "\\quit"}:
            print("ending chat...")
            break

        history.append({"role": "user", "content": user_input})

        stream = ollama.chat(
            model=model,
            messages=history,
            options={
                "temperature": temperature,  # try more randomness
                "num_predict": num_predict,
            },
            stream=True,
        )
        full_chunks = []
        print(f"{model}: ", end="")
        for chunk in stream:
            content = chunk["message"]["content"]
            print(content, end="", flush=True)
            full_chunks.append(content)
        print()

        assistant_message = "".join(full_chunks)
        history.append({"role": "assistant", "content": assistant_message})


if __name__ == "__main__":
    chat_loop()


Starting chat with phi3. Type 'exit' to quit.


You should try this loop implemented in chat_loop.py

Now let's try to add one more thing to our simple loop.. For now we are saving `history` list in RAM. It works, but it doens't work very good. So let's do one more baby step toward building an agent brain. We will log our chat history and put it in appropriate format (JSON) with timestamps. For this exercise we going to use one JSON file per chat session.

What we will log:
- model, temperature, num_predict
- started_at, ended_at
- messages


In [ ]:
from datetime import datetime
import json
import os
import ollama

MODEL_NAME = "gemma3:1b"

def chat_loop(model: str = MODEL_NAME, temperature: float = 1.1, num_predict: int = 512):
    history = []
    print(f"Starting chat with {model}. Type '\\exit', '\\q' or '\\quit' to quit.")
    
    session_start = datetime.now().isoformat
    log = {
        "model": model,
        "temperature": temperature,
        "num_predict": num_predict,
        "started_at": session_start,
        "messages": [],
    }
    
    try:
        while True:
            try:
                user_input = input("> ")
            except (KeyboardInterrupt, EOFError):
                print("\nerror, ending chat...")
                break
            
            if user_input.strip().lower() in {"\\exit", "\\q", "\\quit"}:
                print("\n\quit command, ending chat...")
                break
            
            now = datetime.now().isoformat()
            
            user_msg = {"role": "user", "content": user_input}
            history.append(user_msg)
            log["messages"].append(
                {
                    "role": "user",
                    "content": user_input,
                    "timestamp": now,
                }
            )
            
            stream = ollama.chat(
                model=model,
                messages=history,
                options={
                    "temperature": temperature,  # try more randomness
                    "num_predict": num_predict,
                },
                stream=True,
            )
            full_chunks = []
            print(f"{model}: ", end="")
            for chunk in stream:
                content = chunk["message"]["content"]
                print(content, end="", flush=True)
                full_chunks.append(content)
            print()
            
            assistant_message = "".join(full_chunks)
            assistant_msg = {"role": "assistant", "content": assistant_message}
            history.append(assistant_msg)
            
            now = datetime.now().isoformat()
            
            log["messages"].append(
                {
                    "role": "assistant",
                    "content": assistant_message,
                    "timestamp": now,
                }
            )
            
    finally:
        log["ended_at"] = datetime.now().isoformat()
        os.makedirs("chat_logs", exist_ok=True)
        
        safe_start = session_start.replace(":", "-")
        log_path = os.path.join("chat_logs", f"chat_{safe_start}.json")
        
        with open(log_path, "w", encoding="utf-8") as f:
            json.dump(log, f, ensure_ascii=False, indent=4)
            
        print(f"chat log saved to {log_path}")